# Correlation analysis between GPAQ and Fitbit data regarding PA
**[Log 20.10.2023]**  
reprise apres double check du flowchart  

**[Log 16.10.2023]**  
Reduction a 2 methodes  

**[Log 04.10.2023]**  
Suppression de
- BEDREST
- VOC
- AdverseEvent

**[Log 15.08.2023]**  
Nouvelle analyse pour la difference au cours du temps = correlation sur (T3-T1)  
Il faut reprendre toute l'analyse **en ne considerant que des bouts de 10 min pour VPA & MPA**

**[Log 04.07.2023]**  
Il faut refaire l'analyse:
- en ne prenant cette fois les donnees de Fitbit que autour des V1 / V3
- faire une sous analyse par groupes
- faire une sous analyse par V  
  
Pour ca, il faut prendre les fichiers PApain et pas seulement PA
  
**[Log past]**  
We will use this table in our correlation analysis, using each and every subjects in v1 scores of VPA, MVPA and PAtot obtained with GPAQ. And we will compare this data to those obtained with Fitibit. Then we'll realise a Pearson / Spearman test, and a linear regression model for the plot.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import scipy
from scipy import stats
import re
import glob
import shutil
from datetime import date

# 1 - GPAQ
Rappel: sujets 038v3 et 046v1 supprimes (criteres ONAPS)

In [2]:
concat_GPAQ = pd.read_csv('/Users/mgg/Library/CloudStorage/GoogleDrive-matthieu.gallou.guyot@gmail.com/.shortcut-targets-by-id/1WxK2x1j2i3mI-QbxfSsy98PtnJYzUPEH/Dakar MGG/Clean Data GPAQ/Concat files/concatenated2.csv')
concat_GPAQ = concat_GPAQ.drop(columns=['Unnamed: 0']) # suppress columns 'Unnamed: 0'  
concat_GPAQ = concat_GPAQ[['ID', 'Group', 'V','sed', 'VPA','MVPA', 'PAtot']]
concat_GPAQ = concat_GPAQ.rename(columns={'MVPA': 'MPA', 'PAtot': 'MVPA'})
concat_GPAQ

,ID,Group,V,sed,VPA,MPA,MVPA
0,001,S,3,5460.0,0.0,1180.0,1180.0
1,001,S,1,2100.0,0.0,2160.0,2160.0
2,002,S,1,4200.0,720.0,880.0,1600.0
3,003,S,3,8820.0,0.0,1120.0,1120.0
4,003,S,1,1260.0,0.0,1200.0,1200.0
...,...,...,...,...,...,...,...
152,C16,C,1,3780.0,0.0,340.0,340.0
153,C17,C,1,4200.0,240.0,1120.0,1360.0
154,C18,C,1,3780.0,480.0,4920.0,5400.0
155,C19,C,1,4200.0,0.0,3960.0,3960.0


In [3]:
from scipy.stats import kstest

#creating masks
mask_subjects = concat_GPAQ['Group'] == 'S'
mask_V1 = concat_GPAQ['V'] == 1
mask_V3 = concat_GPAQ['V'] == 3

# summerizinf for subjects, V1+V3
df_filtered_S = concat_GPAQ.loc[mask_subjects]

df_filtered_s_V1 = df_filtered_S.loc[df_filtered_S['V'] ==1]
df_filtered_s_V3 = df_filtered_S.loc[df_filtered_S['V'] ==3]

GPAQ_describe_v1 = df_filtered_s_V1[['MPA','VPA', 'MVPA']].describe().round(2)
GPAQ_describe_v3 = df_filtered_s_V3[['MPA','VPA', 'MVPA']].describe().round(2)

In [4]:
print(len(df_filtered_s_V1)), print(len(df_filtered_s_V3))

97
40


(None, None)

In [5]:
from scipy.stats import shapiro
import pandas as pd

# Supposons que df_filtered_s_V1 contient votre DataFrame
variables_to_test = ['MPA', 'VPA', 'MVPA']

# Boucle à travers les variables et effectuer le test de Shapiro-Wilk
print('Distribution GPAQ V1')
for variable in variables_to_test:
    stat, p_value = shapiro(df_filtered_s_V1[variable])
    is_normal = "Normal" if p_value > 0.05 else "Non normal"
    print(f"{variable}: p-value = {p_value:.4f}, Distribution = {is_normal}")
print('')
print('Distribution GPAQ V3')
for variable in variables_to_test:
    stat, p_value = shapiro(df_filtered_s_V3[variable])
    is_normal = "Normal" if p_value > 0.05 else "Non normal"
    print(f"{variable}: p-value = {p_value:.4f}, Distribution = {is_normal}")    

Distribution GPAQ V1
MPA: p-value = 0.0000, Distribution = Non normal
VPA: p-value = 0.0000, Distribution = Non normal
MVPA: p-value = 0.0000, Distribution = Non normal

Distribution GPAQ V3
MPA: p-value = 0.0000, Distribution = Non normal
VPA: p-value = 0.0000, Distribution = Non normal
MVPA: p-value = 0.0000, Distribution = Non normal


---------
# 2 - Fitbit

In [6]:
path_select = '/Users/mgg/Library/CloudStorage/GoogleDrive-matthieu.gallou.guyot@gmail.com/.shortcut-targets-by-id/1WxK2x1j2i3mI-QbxfSsy98PtnJYzUPEH/Dakar MGG/Day selection PA/SelectedDays_PApain'

dict_select = {}

for file_name in os.listdir(path_select):
    if file_name.endswith('.csv') and not file_name.startswith("~$"):
        file_path = os.path.join(path_select, file_name)
        key = file_name[:3]
        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.set_index('Date')
        dict_select[key] = df

In [7]:
path_min = '/Users/mgg/Library/CloudStorage/GoogleDrive-matthieu.gallou.guyot@gmail.com/.shortcut-targets-by-id/1WxK2x1j2i3mI-QbxfSsy98PtnJYzUPEH/Dakar MGG/Clean Data PA/Clean 20230822/min'

data_dict = {}

for file_name in os.listdir(path_min):
    if file_name.endswith('.csv'):
        file_path = os.path.join(path_min, file_name)
        key = file_name[:3]
        df = pd.read_csv(file_path)
        df['ActivityMinute'] = pd.to_datetime(df['ActivityMinute'])
        df = df.set_index('ActivityMinute')
        id_col = df.pop('ID')
        df.insert(0, 'ID', id_col)
        data_dict[key] = df

### Method 1: no data treatment - PAtot = reelle
Excluding LPA (under 3 METs)

In [8]:
resampled_dict1 = {}

for key, df in data_dict.items():
    df['PAtot'] = df['METs'] >= 3
    df['MPA'] = (df['METs'] >= 3) & (df['METs'] <= 5.9)
    df['VPA'] = df['METs'] >= 6

    daily_resampled_PAtot = df[df['PAtot']]['METs'].resample('D').sum()
    daily_resampled_MPA = df[df['MPA']]['METs'].resample('D').sum()
    daily_resampled_VPA = df[df['VPA']]['METs'].resample('D').sum()

    resampled_dict1[key] = pd.DataFrame({
        'PAtot_1': daily_resampled_PAtot,
        'MPA_1': daily_resampled_MPA,
        'VPA_1': daily_resampled_VPA
    })


### Method 2 : using GPAQ criteria (MET) on Fitbit data, with 10 min bouts
- under 1.5 MET : sed
- between 4 and 8 MET : MVPA
- over 8 : VPA
- over 4 MET : PAtot ( different from MVPA + VPA). **This is highly discutable! Not considering LVPA (1.5 - 4 MET)**

In [9]:
def filter_dataframe(df):
    df.index = pd.to_datetime(df.index)
    df['time_diff'] = df.index.to_series().diff()
    group_series = (df['time_diff'] > pd.Timedelta(minutes=10)).cumsum()
    filtered_df = df.groupby(group_series).filter(lambda group: group.index.size >= 10)
    filtered_df = filtered_df.drop(columns=['time_diff'])
    return filtered_df

def resample_and_sum(df):
    df.index = pd.to_datetime(df.index)
    resampled_df = df['MET.min'].resample('D').sum()
    return resampled_df

resampled_dict2 = {}

for key, df in data_dict.items():
    df['MPA'] = (df['METs'] >= 4) & (df['METs'] <= 7.9)
    df['VPA'] = df['METs'] >= 8
    df['PAtot'] = df['METs'] >= 4

    df['MET.min'] = 0
    df.loc[(df['METs'] >= 4) & (df['METs'] <= 7.9), 'MET.min'] = 4
    df.loc[df['METs'] >= 8, 'MET.min'] = 8

    df_MPA = df[df['MPA'] == True]
    df_VPA = df[df['VPA'] == True]
    df_PAtot = df[df['PAtot'] == True]

    filtered_df_PAtot = filter_dataframe(df_PAtot)
    filtered_df_MPA = filter_dataframe(df_MPA)
    filtered_df_VPA = filter_dataframe(df_VPA)

    if filtered_df_PAtot.index.size >= 10 or filtered_df_MPA.index.size >= 10 or filtered_df_VPA.index.size >= 10:
        resampled_PAtot = resample_and_sum(filtered_df_PAtot)
        resampled_MPA = resample_and_sum(filtered_df_MPA)
        resampled_VPA = resample_and_sum(filtered_df_VPA)

        full_index = pd.date_range(start=min(resampled_PAtot.index.min(), resampled_MPA.index.min(), resampled_VPA.index.min()),
                                   end=max(resampled_PAtot.index.max(), resampled_MPA.index.max(), resampled_VPA.index.max()),
                                   freq='D')
        result_df = pd.DataFrame(index=full_index)
        result_df['PAtot_basedOnMETS_2'] = resampled_PAtot
        result_df['MPA_basedOnMETS_2'] = resampled_MPA
        result_df['VPA_basedOnMETS_2'] = resampled_VPA
        result_df = result_df.fillna(0)
        result_df['USABLE_basedOnMETS_2'] = result_df['PAtot_basedOnMETS_2'].apply(lambda x: x > 0)
    
        resampled_dict2[key] = result_df


/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

...

/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_diff'] = df.index.to_series().diff()
/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/1252269026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

# Gather

In [10]:
full_concat_fitbit = {}
full_concat_fitbit_v1 = {}
full_concat_fitbit_v3 = {}
clean_concat_fitbit_v1 = {}
clean_concat_fitbit_v3 = {}

columns_to_join = ['RECORDS_m', 'WEAR_hand', 'DATA_OK_FITBIT',
                   'BEDREST', 'VOC_period', 'AdverseEvent',
                   'Date_V0', 'Date_V1', 'Date_V2', 'Date_V3',
#                    'Groupe', 
                   'V', 'ID']


common_keys = set(dict_select.keys()) & set(resampled_dict1.keys()) & set(resampled_dict2.keys())

# Initialiser les compteurs
total_raws = 0
removed_rows_condition1_total = 0


for key in common_keys:
    df = dict_select[key]
    df1 = resampled_dict1[key]
    df2 = resampled_dict2[key]

    df3 = df2.join(df1)
    df5 = df3.join(df[columns_to_join])

    full_concat_fitbit[key] = df3
    
    df3_filtered_v1 = df5[df5['V'] ==1]
    df3_filtered_v3 = df5[df5['V'] ==3]
    full_concat_fitbit_v1[key] = df3_filtered_v1
    full_concat_fitbit_v3[key] = df3_filtered_v3

    df_filtered = df5.loc[
        (df5['RECORDS_m'] == True) & 
        (df5['WEAR_hand'] == True) & 
#         (df5['BEDREST'] == False) & 
        (df5['VOC_period'] == False) & 
        (df5['AdverseEvent'] == False) 
        & (df5['DATA_OK_FITBIT'] == True)
    ]

    # Mettre à jour les compteurs
    total_raws += len(df5)
    removed_rows_condition1_total += len(df5) - len(df_filtered)
    
    df_selected_v1 = df_filtered[df_filtered['V'] == 1]#.tail(7)
    df_selected_v3 = df_filtered[df_filtered['V'] == 3]#.tail(7) #finaly considering all period


    clean_concat_fitbit_v1[key] = df_selected_v1
    if not df_selected_v3.empty:
        clean_concat_fitbit_v3[key] = df_selected_v3

# Imprimer les résultats totaux
print(f"Nombre total de lignes : {total_raws}")
print(f"Nombre total de lignes éliminées par (df5['BEDREST'] == False) & (df5['VOC_period'] == False) & (df5['AdverseEvent'] == False) : {removed_rows_condition1_total}")

Nombre total de lignes : 7939
Nombre total de lignes éliminées par (df5['BEDREST'] == False) & (df5['VOC_period'] == False) & (df5['AdverseEvent'] == False) : 1744


In [11]:
fullCrassev1 = pd.concat(full_concat_fitbit_v1.values())
fullCrassev3 = pd.concat(full_concat_fitbit_v3.values())
fullProprev1 = pd.concat(clean_concat_fitbit_v1.values())
fullProprev3 = pd.concat(clean_concat_fitbit_v3.values())
print(f"Len {len(fullCrassev1)}, Len {len(fullProprev1)}")
print(f"Len {len(fullCrassev3)}, Len {len(fullProprev3)}")

Len 4362, Len 3324
Len 1476, Len 1334


In [12]:
data = []

all_keys = set(full_concat_fitbit_v1.keys()) | set(clean_concat_fitbit_v1.keys()) | set(full_concat_fitbit_v3.keys()) | set(clean_concat_fitbit_v3.keys())

for key in all_keys:
    len_full_v1 = len(full_concat_fitbit_v1.get(key, []))
    len_clean_v1 = len(clean_concat_fitbit_v1.get(key, []))
    len_full_v3 = len(full_concat_fitbit_v3.get(key, []))
    len_clean_v3 = len(clean_concat_fitbit_v3.get(key, []))
    
    data.append({
        'Key': key,
        'len_full_v1': len_full_v1,
        'len_clean_v1': len_clean_v1,
        'len_full_v3': len_full_v3,
        'len_clean_v3': len_clean_v3
    })

len_df = pd.DataFrame(data)
len_df.rename(columns={
    'Key': 'ID',
    'len_full_v1': 'N days full v1',
    'len_clean_v1': 'N days clean v1',
    'len_full_v3': 'N days full v3',
    'len_clean_v3': 'N days clean v3'
}, inplace=True)
len_df.sort_values(by='ID', inplace=True)
len_df.drop_duplicates(subset='ID', keep='first', inplace=True)
len_df.reset_index(drop=True, inplace=True)

len_df


,ID,N days full v1,N days clean v1,N days full v3,N days clean v3
0,001,42,37,38,38
1,002,27,26,0,0
2,003,42,30,48,48
3,004,35,31,41,41
4,005,50,40,35,34
...,...,...,...,...,...
93,104,35,29,28,27
94,105,47,46,21,21
95,106,40,37,0,0
96,108,24,19,24,24


In [13]:
pd.options.display.max_rows = None
len_df

,ID,N days full v1,N days clean v1,N days full v3,N days clean v3
0,001,42,37,38,38
1,002,27,26,0,0
2,003,42,30,48,48
3,004,35,31,41,41
4,005,50,40,35,34
5,006,38,23,0,0
6,007,38,35,0,0
7,008,37,22,0,0
8,009,38,30,0,0
9,010,53,19,0,0


In [14]:
x = 10
filtered_len_df_v1 = len_df[len_df['N days clean v1'] >= x]
filtered_len_df_v3 = len_df[len_df['N days clean v3'] >= x]
filtered_len_df_delta = len_df[(len_df['N days clean v1'] >= x) & (len_df['N days clean v3'] >= x)]

print(f"Nombre de sujets a V1 = {len(clean_concat_fitbit_v1)}")
print(f"Nombre de sujets a V1 avec {x} jours au moins = {len(filtered_len_df_v1)}")
print('---')
print(f"Nombre de sujets a V3 = {len(clean_concat_fitbit_v3)}")
print(f"Nombre de sujets a V3 avec {x} jours au moins = {len(filtered_len_df_v3)}")
print('---')
print(f"Nombre de sujets avec {x} jours au moins a V1 et V3 (delta) = {len(filtered_len_df_delta)}")


Nombre de sujets a V1 = 98
Nombre de sujets a V1 avec 10 jours au moins = 97
---
Nombre de sujets a V3 = 41
Nombre de sujets a V3 avec 10 jours au moins = 41
---
Nombre de sujets avec 10 jours au moins a V1 et V3 (delta) = 41


## 3. Calculate the mean over a week
Using different methods

In [15]:
############## RESAMPLING - calculate the MET.min.week with different methods
df = clean_concat_fitbit_v1['090']

# 1- mean per week, using .resample, not using missing dates
df_resampled = df.resample('W', loffset='1D').sum()

# 2- mean per week, using .resample, excluding weeks with missing dates
df_resampled2 = df.resample('W').sum()

# 3- overall mean, using the all period
df_resampled3 = (df.sum()/(len(df.index)))*7

# 4- oeverall mean, using all weeks
df_resampled4 = df_resampled.sum()/(len(df_resampled.index))

# 5- moyenne des 7 jours deja selectionnes
df_resampled5 = df.mean()

# 6 - cumul sur la semaine
df_resampled6 = df.sum()

df_resampled3

/var/folders/24/kprm3l051vg4589bhtmv1_mw0000gn/T/ipykernel_2891/975066783.py:5: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to_offset
>>> df = df.resample(freq="3s").mean()
>>> df.index = df.index.to_timestamp() + to_offset("8H")

  df_resampled = df.resample('W', loffset='1D').sum()


PAtot_basedOnMETS_2     2314.666667
MPA_basedOnMETS_2       2190.222222
VPA_basedOnMETS_2         35.259259
USABLE_basedOnMETS_2       6.740741
PAtot_1                 5295.344444
MPA_1                   4267.588889
VPA_1                   1027.755556
RECORDS_m                       7.0
WEAR_hand                       7.0
DATA_OK_FITBIT                  7.0
BEDREST                    0.259259
VOC_period                      0.0
AdverseEvent                    0.0
Date_V0                         0.0
Date_V1                         0.0
Date_V2                         0.0
Date_V3                         0.0
V                               7.0
ID                            630.0
dtype: object

In [16]:
data = []

all_keys = set(full_concat_fitbit.keys()) | set(clean_concat_fitbit_v1.keys()) | set(clean_concat_fitbit_v3.keys())

for key in all_keys:
    len_full = len(full_concat_fitbit.get(key, [])) + len(clean_concat_fitbit_v1.get(key, []))
    len_clean_v1 = len(clean_concat_fitbit_v1.get(key, []))
    len_clean_v3 = len(clean_concat_fitbit_v3.get(key, []))
    
    data.append({'Key': key, 'len_full': len_full, 'len_clean_v1': len_clean_v1, 'len_clean_v3': len_clean_v3})

len_df = pd.DataFrame(data)
len_df.rename(columns={
    'Key': 'ID',
    'len_full': 'N days full',
    'len_clean_v1': 'N days clean v1',
    'len_clean_v3': 'N days clean v3'
}, inplace=True)
len_df.sort_values(by='ID', inplace=True)
len_df.drop_duplicates(subset='ID', keep='first', inplace=True)
len_df.reset_index(drop=True, inplace=True)

len_df

,ID,N days full,N days clean v1,N days clean v3
0,001,173,37,38
1,002,72,26,0
2,003,162,30,48
3,004,170,31,41
4,005,169,40,34
5,006,62,23,0
6,007,73,35,0
7,008,59,22,0
8,009,69,30,0
9,010,73,19,0


In [17]:
df_v1 = len_df[len_df['N days clean v1'] != 0]
df_v3 = len_df[len_df['N days clean v3'] != 0]

print("\nDescription pour N days clean v1 (sans les 0) :")
print(df_v1['N days clean v1'].describe())

print("\nDescription pour N days clean v3 (sans les 0) :")
print(df_v3['N days clean v3'].describe())


Description pour N days clean v1 (sans les 0) :
count    98.000000
mean     33.918367
std       9.515775
min       6.000000
25%      27.000000
50%      34.000000
75%      40.000000
max      55.000000
Name: N days clean v1, dtype: float64

Description pour N days clean v3 (sans les 0) :
count    41.000000
mean     32.536585
std       9.975213
min      12.000000
25%      26.000000
50%      34.000000
75%      40.000000
max      48.000000
Name: N days clean v3, dtype: float64


df_resampled3 semble la plus precise actuellement. Nous utiliserons donc un cumul sur toute la periode, rapporte a une duree de 7 jours.

# 3 - GPAQ & FITBIT
## 3.1. Only at V1, only subjects, all groups together

In [18]:
results = []

for key, df in clean_concat_fitbit_v1.items():
    df = df.drop([
#         'USABLE_basedOnINT_2a', 'USABLE_basedOnINT_2b', 'USABLE_basedOnMETS_1a', 'USABLE_basedOnMETS_1b', 
                  'RECORDS_m', 'WEAR_hand', 'DATA_OK_FITBIT', 'V', 
#                   'Groupe'
                 ], axis=1)
    
    # Sélectionner seulement les colonnes numériques pour la somme
    numeric_columns = df.select_dtypes(include=[np.number])
    avg_sum = (numeric_columns.sum() / len(df.index)) * 7
    
    avg_sum['ID'] = key
    results.append(avg_sum)

result_v1 = pd.DataFrame(results)
result_v1.set_index('ID', inplace=True)
result_v1 = result_v1.sort_index()
result_v1 = result_v1.round(2)

mask_subjects = concat_GPAQ['Group'] == 'S'
mask_V1 = concat_GPAQ['V'] == 1
concat_GPAQ_S_V1 = concat_GPAQ.loc[mask_subjects & mask_V1]
concat_GPAQ_S_V1.set_index('ID', inplace=True)
concat_GPAQ_S_V1 = concat_GPAQ_S_V1.sort_index()

resultv1 = result_v1.join(concat_GPAQ_S_V1, how='inner', lsuffix='_fitbit', rsuffix='_GPAQ')

liste_ID_a_exclure = ['051'] 
resultv1 = resultv1[~resultv1.index.isin(liste_ID_a_exclure)]

print(len(resultv1))
resultv1

96


,PAtot_basedOnMETS_2,MPA_basedOnMETS_2,VPA_basedOnMETS_2,PAtot_1,MPA_1,VPA_1,Group,V,sed,VPA,MPA,MVPA
ID,,,,,,,,,,,,
001,809.73,809.73,0.00,3402.11,2134.17,1267.95,S,1,2100.0,0.0,2160.0,2160.0
002,1336.46,1336.46,0.00,3382.75,1839.20,1543.55,S,1,4200.0,720.0,880.0,1600.0
003,357.47,351.87,0.00,2172.50,2064.95,107.54,S,1,1260.0,0.0,1200.0,1200.0
004,580.77,580.77,0.00,3604.46,3093.73,510.73,S,1,1260.0,0.0,1740.0,1740.0
005,492.80,492.80,0.00,3608.92,2938.48,670.44,S,1,5040.0,0.0,280.0,280.0
006,591.65,591.65,0.00,4452.24,3620.31,831.93,S,1,840.0,320.0,1040.0,1360.0
007,3431.20,3416.80,0.00,9151.16,4364.20,4786.96,S,1,420.0,0.0,1440.0,1440.0
008,3087.64,3087.64,0.00,6906.42,4070.66,2835.76,S,1,5460.0,0.0,2880.0,2880.0
009,1001.47,971.60,0.00,4699.99,3396.05,1303.94,S,1,4620.0,960.0,4560.0,5520.0


In [19]:
from scipy.stats import shapiro
import pandas as pd

variables_to_test = ['PAtot_1', 'MPA_1', 'VPA_1',
                     'PAtot_basedOnMETS_2','MPA_basedOnMETS_2','VPA_basedOnMETS_2',
                     'MPA', 'VPA', 'MVPA']

# Boucle à travers les variables et effectuer le test de Shapiro-Wilk
print('Distribution des valeurs brutes (1), avec bouts & 4-8 (2) & GPAQ V1')
for variable in variables_to_test:
    stat, p_value = shapiro(resultv1[variable])
    is_normal = "Normal" if p_value > 0.05 else "Non normal"
    print(f"{variable}: p-value = {p_value:.4f}, Distribution = {is_normal}")

Distribution des valeurs brutes (1), avec bouts & 4-8 (2) & GPAQ V1
PAtot_1: p-value = 0.0000, Distribution = Non normal
MPA_1: p-value = 0.0000, Distribution = Non normal
VPA_1: p-value = 0.0000, Distribution = Non normal
PAtot_basedOnMETS_2: p-value = 0.0000, Distribution = Non normal
MPA_basedOnMETS_2: p-value = 0.0000, Distribution = Non normal
VPA_basedOnMETS_2: p-value = 0.0000, Distribution = Non normal
MPA: p-value = 0.0000, Distribution = Non normal
VPA: p-value = 0.0000, Distribution = Non normal
MVPA: p-value = 0.0000, Distribution = Non normal


In [20]:
tableDescribeV1 = resultv1.describe().round(2)
tableDescribeV1

,PAtot_basedOnMETS_2,MPA_basedOnMETS_2,VPA_basedOnMETS_2,PAtot_1,MPA_1,VPA_1,V,sed,VPA,MPA,MVPA
count,96.00,96.00,96.00,96.00,96.00,96.00,96.0,96.00,96.00,96.00,96.00
mean,2441.16,2336.53,45.50,6268.02,4578.65,1689.37,1.0,3181.35,448.33,3288.33,3736.67
std,2198.29,2096.68,119.60,3448.81,2666.76,1377.81,0.0,1654.26,1355.07,4624.99,4842.43
min,111.00,111.00,0.00,1507.83,1073.10,37.78,1.0,105.00,0.00,0.00,0.00
25%,994.12,943.44,0.00,3831.60,2630.06,775.68,1.0,2100.00,0.00,900.00,900.00
50%,1719.48,1627.38,0.00,5179.37,3829.86,1329.16,1.0,3360.00,0.00,1470.00,1680.00
75%,3280.13,3139.40,28.00,8047.00,6125.28,2112.35,1.0,4200.00,0.00,3360.00,4080.00
max,11109.00,10988.25,809.73,18165.50,13689.72,6960.66,1.0,8820.00,9600.00,24480.00,24480.00


In [21]:
# result1.to_csv('/Users/mgg/Library/CloudStorage/GoogleDrive-matthieu.gallou.guyot@gmail.com/.shortcut-targets-by-id/1WxK2x1j2i3mI-QbxfSsy98PtnJYzUPEH/Dakar MGG/Analysis GPAQ-Fitbit/VPA MVPA Fitbit GPAQ_V1.csv')
# tableDescribeV1.to_csv('/Users/mgg/Library/CloudStorage/GoogleDrive-matthieu.gallou.guyot@gmail.com/.shortcut-targets-by-id/1WxK2x1j2i3mI-QbxfSsy98PtnJYzUPEH/Dakar MGG/Analysis GPAQ-Fitbit/describe_V1.csv')

In [22]:
from scipy.stats import spearmanr

cols_fitbit = [
    'PAtot_1', 'MPA_1', 'VPA_1',
    'PAtot_basedOnMETS_2', 'MPA_basedOnMETS_2', 'VPA_basedOnMETS_2'
    ]

cols_GPAQ = ['MVPA', 'MPA', 'VPA']

print('Correlation Spearman at T1: is GPAQ equal to Fitbit to mesure PA ?')
print('')
print('Method 1 = Raw fitbit data')
for i in range(3):
    # Drop rows with NaN values from both columns before calculating correlation
    clean_data = resultv1[[cols_fitbit[i], cols_GPAQ[i]]].dropna()
    corr, p_value = spearmanr(clean_data[cols_fitbit[i]], clean_data[cols_GPAQ[i]])
    print(f"{cols_fitbit[i]} vs {cols_GPAQ[i]} : corr = {corr:.2f}, p-value = {p_value:.4f}")
print(f"N={len(clean_data)}")
print(' ')

print('Method 2 = Traduction of Fitbit data into scores based on GPAQs MET definition (with bouts, 4-8 METs)')
for i in range(3):
    # Drop rows with NaN values from both columns before calculating correlation
    clean_data = resultv1[[cols_fitbit[i+3], cols_GPAQ[i]]].dropna()
    corr, p_value = spearmanr(clean_data[cols_fitbit[i+3]], clean_data[cols_GPAQ[i]])
    print(f"{cols_fitbit[i+3]} vs {cols_GPAQ[i]} : corr = {corr:.2f}, p-value = {p_value:.4f}")
print(f"N={len(clean_data)}")
print(' ')

Correlation Spearman at T1: is GPAQ equal to Fitbit to mesure PA ?

Method 1 = Raw fitbit data
PAtot_1 vs MVPA : corr = 0.07, p-value = 0.5200
MPA_1 vs MPA : corr = 0.03, p-value = 0.7568
VPA_1 vs VPA : corr = 0.08, p-value = 0.4273
N=96
 
Method 2 = Traduction of Fitbit data into scores based on GPAQs MET definition (with bouts, 4-8 METs)
PAtot_basedOnMETS_2 vs MVPA : corr = 0.03, p-value = 0.7624
MPA_basedOnMETS_2 vs MPA : corr = 0.08, p-value = 0.4418
VPA_basedOnMETS_2 vs VPA : corr = 0.08, p-value = 0.4405
N=96
 


## 3.2. Only subjects, V3

In [23]:
results3 = []

for key, df in clean_concat_fitbit_v3.items():
    df = df.drop([
#         'USABLE_basedOnINT_2a', 'USABLE_basedOnINT_2b', 'USABLE_basedOnMETS_1a', 'USABLE_basedOnMETS_1b', 
        'RECORDS_m', 'WEAR_hand', 'DATA_OK_FITBIT', 'V', 
#                   'Groupe'
                 ], axis=1)
    
    # Sélectionner seulement les colonnes numériques pour la somme
    numeric_columns = df.select_dtypes(include=[np.number])
    avg_sum = (numeric_columns.sum() / len(df.index)) * 7
    
    avg_sum['ID'] = key
    results3.append(avg_sum)

result_v3 = pd.DataFrame(results3)
result_v3.set_index('ID', inplace=True)
result_v3 = result_v3.sort_index()
result_v3 = result_v3.round(2)

mask_subjects = concat_GPAQ['Group'] == 'S'
mask_V3 = concat_GPAQ['V'] == 3
concat_GPAQ_S_V3 = concat_GPAQ.loc[mask_subjects & mask_V3]
concat_GPAQ_S_V3.set_index('ID', inplace=True)
concat_GPAQ_S_V3 = concat_GPAQ_S_V3.sort_index()

resultv3 = result_v3.join(concat_GPAQ_S_V3, how='inner', lsuffix='_fitbit', rsuffix='_GPAQ')

print(len(resultv3))
resultv3

40


,PAtot_basedOnMETS_2,MPA_basedOnMETS_2,VPA_basedOnMETS_2,PAtot_1,MPA_1,VPA_1,Group,V,sed,VPA,MPA,MVPA
ID,,,,,,,,,,,,
001,625.58,625.58,0.00,2970.04,2018.89,951.15,S,3,5460.0,0.0,1180.0,1180.0
003,187.25,187.25,0.00,1824.26,1743.55,80.70,S,3,8820.0,0.0,1120.0,1120.0
004,1316.68,1296.20,0.00,5811.13,4626.08,1185.05,S,3,5880.0,0.0,4680.0,4680.0
005,637.41,637.41,0.00,3861.71,3062.85,798.86,S,3,5460.0,0.0,560.0,560.0
020,873.00,873.00,0.00,4609.80,3632.45,977.35,S,3,6720.0,0.0,1680.0,1680.0
022,2091.20,2091.20,0.00,5905.92,2470.30,3435.62,S,3,1260.0,0.0,7200.0,7200.0
024,1930.53,1285.05,347.79,5117.74,2745.22,2372.52,S,3,3360.0,0.0,1440.0,1440.0
025,1731.88,1583.65,16.47,4690.14,1943.08,2747.07,S,3,4200.0,0.0,960.0,960.0
026,1239.70,1117.90,61.60,4772.78,3185.02,1587.76,S,3,6720.0,8640.0,4880.0,13520.0


In [24]:
tableDescribeV3 = resultv3.describe().round(2)
tableDescribeV3

,PAtot_basedOnMETS_2,MPA_basedOnMETS_2,VPA_basedOnMETS_2,PAtot_1,MPA_1,VPA_1,V,sed,VPA,MPA,MVPA
count,40.00,40.00,40.00,40.00,40.00,40.00,40.0,40.00,40.00,40.00,40.00
mean,2730.80,2576.81,92.42,6798.21,5017.72,1780.50,3.0,3601.50,393.00,2703.00,3096.00
std,2425.31,2384.97,295.55,3676.04,2950.81,1388.79,0.0,1771.44,1579.87,3482.63,4115.28
min,187.25,187.25,0.00,1824.26,1743.55,80.70,3.0,1260.00,0.00,0.00,0.00
25%,1160.20,1069.15,0.00,4577.39,2810.90,943.32,3.0,2520.00,0.00,1080.00,1080.00
50%,1858.04,1681.68,0.00,5798.64,4445.35,1387.48,3.0,3360.00,0.00,1220.00,1220.00
75%,2979.60,2775.14,48.58,7546.96,6275.65,2456.46,3.0,4200.00,0.00,2820.00,2820.00
max,9565.58,9242.60,1488.24,16785.85,11890.62,7019.50,3.0,8820.00,8640.00,14880.00,14880.00


# 4 - Appreciation of the variation over time
Here we are interested in looking at the difference of variation between v1 and v3 for every subjects that goes to v3. We will compare MVPA, VPA and PAtot values variation from Fitbit to GPAQ. To do so, we need to know the distribution of data, then apply a Student T test (if normal) or a Wilcoxon signed rank (if not).

In [25]:
# Liste des colonnes à exclure
excluded_columns = ['Group', 'V']

# Filtrer les lignes de resultv1 pour conserver uniquement les IDs présents dans resultv3
resultv1_filtered = resultv1[resultv1.index.isin(resultv3.index)]

# Calculer la différence entre les colonnes correspondantes
merged_df_delta = resultv3.drop(columns=excluded_columns) - resultv1_filtered.drop(columns=excluded_columns)

# Ajouter le préfixe "D_" aux noms de colonnes
merged_df_delta.columns = "D_" + merged_df_delta.columns

# Réinitialiser l'index pour obtenir 'ID' en tant que colonne
merged_df_delta.reset_index(inplace=True)

merged_df_delta.describe().round(2)

,D_PAtot_basedOnMETS_2,D_MPA_basedOnMETS_2,D_VPA_basedOnMETS_2,D_PAtot_1,D_MPA_1,D_VPA_1,D_sed,D_VPA,D_MPA,D_MVPA
count,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00,40.00
mean,1132.58,1046.48,60.25,2062.85,1370.03,692.83,309.75,231.00,-706.50,-475.50
std,1736.82,1793.48,313.09,2758.73,2369.47,816.17,2365.38,1390.53,4682.18,5100.41
min,-789.55,-699.11,-409.23,-1384.73,-1286.11,-842.08,-5460.00,-2880.00,-18720.00,-18720.00
25%,328.24,181.89,0.00,515.61,198.43,148.89,-840.00,0.00,-1545.00,-1120.00
50%,651.78,521.10,0.00,1167.42,483.73,502.58,0.00,0.00,60.00,120.00
75%,1211.36,998.09,24.00,2125.78,1553.09,1141.23,892.50,0.00,1570.00,1615.00
max,6955.10,7438.99,1488.24,10930.93,10120.66,2421.34,7560.00,6240.00,9480.00,11640.00


In [26]:
from scipy.stats import spearmanr

cols_fitbit = [
    'D_PAtot_1', 'D_MPA_1', 'D_VPA_1',
    'D_PAtot_basedOnMETS_2','D_MPA_basedOnMETS_2', 'D_VPA_basedOnMETS_2'
              ]
cols_GPAQ = ['D_MVPA', 'D_MPA', 'D_VPA']

print("Correlation Spearman for delta (T1-T3): is GPAQ equal to Fitbit to mesure PA variation over time?")
print('')
print('Method 1 = Raw Fitbit Data')
for i in range(3):
    # Drop rows with NaN values from both columns before calculating correlation
    clean_data = merged_df_delta[[cols_fitbit[i], cols_GPAQ[i]]].dropna()
    corr, p_value = spearmanr(clean_data[cols_fitbit[i]], clean_data[cols_GPAQ[i]])
    print(f"{cols_fitbit[i]} vs {cols_GPAQ[i]} : corr = {corr:.2f}, p-value = {p_value:.4f}")
print(f"N={len(clean_data)}")
print(' ')

print('Method 2 = Traduction of Fitbit data into scores based on GPAQs MET definition (with 10 min bouts, 4-8 METs)')
for i in range(3):
    # Drop rows with NaN values from both columns before calculating correlation
    clean_data = merged_df_delta[[cols_fitbit[i+3], cols_GPAQ[i]]].dropna()
    corr, p_value = spearmanr(clean_data[cols_fitbit[i+3]], clean_data[cols_GPAQ[i]])
    print(f"{cols_fitbit[i+3]} vs {cols_GPAQ[i]} : corr = {corr:.2f}, p-value = {p_value:.4f}")
print(f"N={len(clean_data)}")
print(' ')

Correlation Spearman for delta (T1-T3): is GPAQ equal to Fitbit to mesure PA variation over time?

Method 1 = Raw Fitbit Data
D_PAtot_1 vs D_MVPA : corr = 0.13, p-value = 0.4396
D_MPA_1 vs D_MPA : corr = -0.04, p-value = 0.8266
D_VPA_1 vs D_VPA : corr = 0.12, p-value = 0.4456
N=40
 
Method 2 = Traduction of Fitbit data into scores based on GPAQs MET definition (with 10 min bouts, 4-8 METs)
D_PAtot_basedOnMETS_2 vs D_MVPA : corr = 0.18, p-value = 0.2580
D_MPA_basedOnMETS_2 vs D_MPA : corr = -0.01, p-value = 0.9505
D_VPA_basedOnMETS_2 vs D_VPA : corr = 0.04, p-value = 0.7864
N=40
 


In [27]:
stop

NameError: name 'stop' is not defined

# 5 - Sous analyses
## Il y a-t-il une difference entre GPAQ et Fitbit 1a (METs & bouts) a T1 ?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon

# Colonnes à comparer
columns_to_compare = [
    ('PAtot_basedOnMETS_1a', 'PAtot'),
    ('MPA_basedOnMETS_1a', 'MVPA'),
    ('VPA_basedOnMETS_1a', 'VPA')
]

# Réaliser les tests de Wilcoxon et stocker les résultats dans un dictionnaire
wilcoxon_results = {}

for col1, col2 in columns_to_compare:
    stat, p_value = wilcoxon(resultv1[col1], resultv1[col2])
    wilcoxon_results[(col1, col2)] = {'statistic': stat, 'p-value': p_value}

# Créer la représentation graphique avec trois sous-figures
fig, axes = plt.subplots(nrows=1, ncols=len(columns_to_compare), figsize=(15, 5))

for idx, (col1, col2) in enumerate(columns_to_compare):
    ax = axes[idx]

    # Créer les boîtes à moustaches
    box_data = [resultv1[col1], resultv1[col2]]
    ax.boxplot(box_data, labels=[col1, col2])

    # Ajouter l'annotation avec * et la valeur du test et du p
    star_marker = '*' if wilcoxon_results[(col1, col2)]['p-value'] < 0.05 else ''
    annotation = f"{star_marker} p={wilcoxon_results[(col1, col2)]['p-value']:.3f}\n(n={len(resultv1)})"
    ax.annotate(annotation, xy=(0.5, 0.9), xycoords='axes fraction', ha='center', fontsize=12)

    ax.set_title(f"Comparison: {col1} vs {col2}")

plt.tight_layout()
plt.show()


## Il y a-t-il une difference entre GPAQ et Fitbit raw (all active, no bouts)a T1 ?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon

# Colonnes à comparer
columns_to_compare = [
    ('PAtot_3', 'PAtot'),
    ('MPA_3', 'MVPA'),
    ('VPA_3', 'VPA')
]

# Réaliser les tests de Wilcoxon et stocker les résultats dans un dictionnaire
wilcoxon_results = {}

for col1, col2 in columns_to_compare:
    stat, p_value = wilcoxon(resultv1[col1], resultv1[col2])
    wilcoxon_results[(col1, col2)] = {'statistic': stat, 'p-value': p_value}

# Créer la représentation graphique avec trois sous-figures
fig, axes = plt.subplots(nrows=1, ncols=len(columns_to_compare), figsize=(15, 5))

for idx, (col1, col2) in enumerate(columns_to_compare):
    ax = axes[idx]

    # Créer les boîtes à moustaches
    box_data = [resultv1[col1], resultv1[col2]]
    ax.boxplot(box_data, labels=[col1, col2])

    # Ajouter l'annotation avec * et la valeur du test et du p
    star_marker = '*' if wilcoxon_results[(col1, col2)]['p-value'] < 0.05 else ''
    annotation = f"{star_marker} p={wilcoxon_results[(col1, col2)]['p-value']:.3f}\n(n={len(resultv1)})"
    ax.annotate(annotation, xy=(0.5, 0.9), xycoords='axes fraction', ha='center', fontsize=12)

    ax.set_title(f"Comparison: {col1} vs {col2}")

plt.tight_layout()
plt.show()


In [ ]:
# Concaténer tous les DataFrames dans un seul DataFrame
combined_df = pd.concat(data_dict.values(), ignore_index=True)

# Créer la figure avec deux sous-figures
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

# Créer l'histogramme pour la colonne 'Intensity'
axes[0].hist(combined_df['Intensity'], bins=[1.5, 2.5, 3.5], align='mid', rwidth=0.8, alpha=0.7)
axes[0].set_xticks([2, 3])
axes[0].set_xticklabels(['Intensity 2', 'Intensity 3'])
axes[0].set_xlabel('Intensity')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Histogram of Intensity')

# Créer l'histogramme pour la colonne 'METs'
axes[1].hist(combined_df['METs'], bins=10, alpha=0.7)
axes[1].set_xlabel('METs')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Histogram of METs')

# Afficher le nombre de sujets
num_subjects = len(data_dict)
fig.suptitle(f"Combined Histograms for {num_subjects} Subjects", fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# Concaténer tous les DataFrames dans un seul DataFrame
combined_df = pd.concat(data_dict.values(), ignore_index=True)

# Filtrer uniquement les valeurs d'Intensity 1, 2 et 3
filtered_intensity = combined_df[combined_df['Intensity'].isin([1, 2, 3])]

# Créer la figure avec une sous-figure
fig, ax = plt.subplots(figsize=(8, 5))

# Créer l'histogramme pour les valeurs filtrées d'Intensity
hist_data, bins, patches = ax.hist(filtered_intensity['Intensity'], bins=[0.5, 1.5, 2.5, 3.5], align='mid', rwidth=0.8, alpha=0.7)
ax.set_xticks([1, 2, 3])
ax.set_xticklabels(['Intensity 1', 'Intensity 2', 'Intensity 3'])
ax.set_xlabel('Intensity')
ax.set_ylabel('Frequency')
ax.set_title('Histogram of Filtered Intensity Values')

# Ajouter le nombre de sujets (DataFrames)
num_subjects = len(data_dict)
ax.text(0.95, 0.95, f'Number of Subjects: {num_subjects}', transform=ax.transAxes, ha='right', va='top')

# Ajouter les valeurs de fréquence au-dessus des barres
for value, patch in zip(hist_data, patches):
    height = patch.get_height()
    ax.annotate(f'{value}', xy=(patch.get_x() + patch.get_width() / 2, height), xytext=(0, 3),
                textcoords='offset points', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
plt.hist(combined_df['METs'])

---
# DESCRIPTIVE

In [ ]:
#only keep  'ID', Steps', 'Calories', and TRUE for 'RECORDS_m',  'WEAR_hand', 'DATA_OK_FITBIT'

descriptive_dict = {}

for key, df in dict_select.items():
    selected_columns = ['ID', 'V', 'Steps', 'Calories', 'RECORDS_m', 'WEAR_hand', 'DATA_OK_FITBIT']
    selected_df = df[selected_columns]
    
    filtered_df = selected_df[(selected_df['RECORDS_m'] == True) & 
                              (selected_df['WEAR_hand'] == True) & 
                              (selected_df['DATA_OK_FITBIT'] == True)]
        
    descriptive_dict[key] = filtered_df

descriptive_dict = {k: descriptive_dict[k] for k in sorted(descriptive_dict)}
# for key, df in sorted_descriptive_dict.items():
#     print(f"KEY {key} - N days of observation = {len(df)}")

In [ ]:
# Créer un DataFrame pour stocker les longueurs
df_lengths = pd.DataFrame(index=dict_select.keys(), columns=['Len_before_selection_V1', 'Len_before_selection_V3'])

# Parcourir le dictionnaire pour calculer les longueurs
for key, df in dict_select.items():
    # Calculer la longueur lorsque 'V' = 1
    len_v1 = len(df[df['V'] == 1])
    
    # Calculer la longueur lorsque 'V' = 3
    len_v3 = len(df[df['V'] == 3])
    
    # Assigner les valeurs calculées au DataFrame df_lengths
    df_lengths.at[key, 'Len_before_selection_V1'] = len_v1
    df_lengths.at[key, 'Len_before_selection_V3'] = len_v3


In [ ]:
# Créer un DataFrame final pour stocker les moyennes (V1)
df_descriptivev1 = pd.DataFrame(columns=['ID', 'Avg_Steps', 'Avg_Calories', 'Num_Days'])

# Créer un DataFrame final pour stocker les moyennes (V3)
df_descriptivev3 = pd.DataFrame(columns=['ID', 'Avg_Steps', 'Avg_Calories', 'Num_Days'])


# Parcourir descriptive_dict pour calculer les moyennes (V1 et V3)
for key, df in descriptive_dict.items():
    # Filtrer les lignes où la colonne 'V' vaut 1
    filtered_df_v1 = df[df['V'] == 1]
    
    # Calculer les moyennes pour 'Steps' et 'Calories' et arrondir (V1)
    avg_steps_v1 = round(filtered_df_v1['Steps'].mean(), 0)
    avg_calories_v1 = round(filtered_df_v1['Calories'].mean(), 0)
    
    # Filtrer les lignes où la colonne 'V' vaut 3
    filtered_df_v3 = df[df['V'] == 3]
    
    # Calculer les moyennes pour 'Steps' et 'Calories' et arrondir (V3)
    avg_steps_v3 = round(filtered_df_v3['Steps'].mean(), 0)
    avg_calories_v3 = round(filtered_df_v3['Calories'].mean(), 0)
    
    # Ajouter les valeurs arrondies au df_descriptivev1
    df_descriptivev1 = df_descriptivev1.append({'ID': key, 'Avg_Steps': avg_steps_v1, 'Avg_Calories': avg_calories_v1, 'Num_Days': len(filtered_df_v1)}, ignore_index=True)
    
    # Ajouter les valeurs arrondies au df_descriptivev3
    df_descriptivev3 = df_descriptivev3.append({'ID': key, 'Avg_Steps': avg_steps_v3, 'Avg_Calories': avg_calories_v3, 'Num_Days': len(filtered_df_v3)}, ignore_index=True)


df_descriptivev3 = df_descriptivev3[df_descriptivev3['Num_Days'] > 0]


# Trier les DataFrames par ID croissant
df_descriptivev1 = df_descriptivev1.sort_values(by='ID')
df_descriptivev3 = df_descriptivev3.sort_values(by='ID')

# Mettre 'ID' en tant qu'index
df_descriptivev1.set_index('ID', inplace=True)
df_descriptivev3.set_index('ID', inplace=True)


In [ ]:
index_values_descriptivev1 = df_descriptivev1.index.tolist()
column_values_filtered_s_V1 = df_filtered_s_V1['ID'].tolist()
values_in_descriptivev1_not_in_filtered_s_V1 = list(set(index_values_descriptivev1) - set(column_values_filtered_s_V1))
values_in_filtered_s_V1_not_in_descriptivev1 = list(set(column_values_filtered_s_V1) - set(index_values_descriptivev1))

print('V1')
print("Values in DESCRIPTIVE but not in GPAQ:")
print(values_in_descriptivev1_not_in_filtered_s_V1)

print("\nValues in GPAQ but not in DESCRIPTIVE:")
print(values_in_filtered_s_V1_not_in_descriptivev1)
print('---')
print('')
index_values_descriptivev3 = df_descriptivev3.index.tolist()
column_values_filtered_s_V3 = df_filtered_s_V3['ID'].tolist()
values_in_descriptivev3_not_in_filtered_s_V3 = list(set(index_values_descriptivev3) - set(column_values_filtered_s_V3))
values_in_filtered_s_V3_not_in_descriptivev3 = list(set(column_values_filtered_s_V3) - set(index_values_descriptivev3))

print('V3')
print("Values in DESCRIPTIVE but not in GPAQ:")
print(values_in_descriptivev3_not_in_filtered_s_V3)

print("\nValues in GPAQ but not in DESCRIPTIVE:")
print(values_in_filtered_s_V3_not_in_descriptivev3)

print('NE PAS OUBLIER DE LES SUPPRIMER (A LA MAIN)')

In [ ]:
# SAVE
save_path = "/Users/mgg/Library/CloudStorage/GoogleDrive-matthieu.gallou.guyot@gmail.com/.shortcut-targets-by-id/1WxK2x1j2i3mI-QbxfSsy98PtnJYzUPEH/Dakar MGG/Analysis GPAQ-Fitbit/descriptive/"
df_descriptivev1.to_csv(save_path + "df_descriptivev1.csv")
df_descriptivev3.to_csv(save_path + "df_descriptivev3.csv")
df_lengths.to_csv(save_path + "len_before_selection.csv")


In [ ]:
descriptive_clean = pd.read_csv("/Users/mgg/Desktop/Article GPAQ : Fitbit/descriptiveGPAQ_clean.csv", index_col='ID')
descriptive_clean.index = descriptive_clean.index.astype(str).str.zfill(3)

result_dfv1 = descriptive_clean.merge(df_descriptivev1, left_index=True, right_index=True, how='inner')
result_dfv1 = result_dfv1.drop("046")

result_dfv3 = descriptive_clean.merge(df_descriptivev3, left_index=True, right_index=True, how='inner')
result_dfv3 = result_dfv3.drop("038")

result_dfv1

In [ ]:
result_dfv1['Num_Days'] = pd.to_numeric(result_dfv1['Num_Days'], errors='coerce')

result_dfv1.describe(include="all").round(2)


In [ ]:
result_dfv3['Num_Days'] = pd.to_numeric(result_dfv3['Num_Days'], errors='coerce')

result_dfv3.describe().round(2)